# Python Oleg Detection API 

## Введение

Данный ноутбук содержит руководство по использованию **Oleg Detection API** для языка Python, примеры использования различных POST запросов для общения с сервером: взаимодействия с учетной записью и распознавание аудиозаписей.

На данный момент реализовано два режима распознавания:
- **Long Mode** используется для распознавания длинных аудиозаписей (от 5 секунд до 60 минут) и возвращает информацию о каждой фразе с речью (начало, конец, длительность фразы и результаты распознавания);
- **Short Mode** используется для распознавания коротких аудиозаписей (до 15 секунд) и возвращает только результат распознавания (без разметки фраз).

Поддерживаются аудиозаписи в формате .wav с битовой глубиной 16 и с любой частотой дискретизации.

In [1]:
# Импорт необходимых пакетов
from od_client import ODAuthentication
from od_client import ODRecognizer
from od_client import load_wav

- **ODAuthentication** используется в учетную запись для входа по логину и паролю, генерации новых токенов (которые далее используются для распознавания), и получения информации об используемых токенах;

- **ODRecognizer** используется для распознавания аудио с помощью токенов.

## Взаимодействия с учетной записью

### Вход в систему

In [2]:
# предоставленные данные для входа
# это пример! пожалуйста, вставьте свои данные
USERNAME = 'username'
PASSWORD = 'password'

In [3]:
# аутентификация
authentificator = ODAuthentication(USERNAME, PASSWORD)

Successfully signed-in with username username


### Получение нового API-токена

Метод **new_token()** используется для генерации нового API-токена, который действует один месяц (максимальное количество действующих токенов для одного пользователя — 10)

In [4]:
token, expire_date = authentificator.new_token()

Successfully received a new token: 8bb9f0bd922e2241551faacb17ac8e8493dc3255f57f4d8f779bf2a9aff38239 that expires 2022-07-22


### Получение информации о действующих токенах

Получение информации о действующих токенах, прикрепленных к пользователю с помощью метода **all_tokens()**

In [5]:
authentificator.all_tokens()

Successfully received all tokens


[{'token': 'eb0c829d19ecff85d02cd1ffa55695536bc94aa162d5338e956e11672ec8cd3c',
  'expires': '2022-07-22'},
 {'token': '8bb9f0bd922e2241551faacb17ac8e8493dc3255f57f4d8f779bf2a9aff38239',
  'expires': '2022-07-22'}]

### Получение информации о количестве распознанных секунд

Получение информации о количестве распознанных секунд пользователем для каждого режима (Long Mode и Short Mode) за весь период с помощью **recognized_seconds_by_user(date='all')**

In [6]:
authentificator.recognized_seconds_by_user(date='all')

Successfully received the number of recognized seconds for all period:
For Short Mode: 0.0
For Long Mode: 0.0
Overall: 0.0


{'short_mode': 0.0, 'long_mode': 0.0, 'all': 0.0}

Получение информации о количестве распознанных секунд пользователем с начала месяца с помощью **recognized_seconds_by_user(date='month')**

In [7]:
authentificator.recognized_seconds_by_user(date='month')

Successfully received the number of recognized seconds for the last month:
For Short Mode: 0.0
For Long Mode: 0.0
Overall: 0.0


{'short_mode': 0.0, 'long_mode': 0.0, 'all': 0.0}

## Распознавание аудиозаписей с помощью API-токена

Получение доступа к системе распознавания с помощью API-токена API_TOKEN (сгенерирован ранее)

In [8]:
# это пример! пожалуйста, вставьте свой токен
API_TOKEN = 'f4cddd608e5e1a7f85cf38e7a79fc2ee532b30f7ce7acf54f51770de41149f20'

In [9]:
recognizer = ODRecognizer(API_TOKEN)

Token is valid


### Получение информации об API-токене

Метод **token_information()** возвращает информацию о владельце токена и период действия

In [10]:
recognizer.token_information()

Successfully received information about the token
Owner: example_username
Expires: 2022-07-22


{'owner': 'example_username', 'expires': '2022-07-22'}

### Получение информации о количестве распознанных секунд для конкретного API-токена

Получение информации о количестве распознанных секунд с помощью токена

In [11]:
recognizer.recognized_seconds_by_token()

Successfully received the number of recognized seconds for the token:
For Short Mode: 166.4
For Long Mode: 555.41
Overall: 721.81


{'short_mode': 166.4, 'long_mode': 555.41, 'all': 721.81}

### Примеры распознавания

Для загрузки байтов из аудиофайла и другой необходимой информации используется предложенный метод **load_wav(path_to_file, channel)**, где:
- **path_to_file** — путь до аудиофайла;
- **max_duration** — количество секунд от начала аудиофайла, которое необходимо загрузить (если max_duration = - 1, то загрузится полный аудиофайл);
- **channel** — номер канала, на котором возможно появление Ассистента Олега (для исходящих звонков со стороны колл-центров чаще всего этот канал под индексом 0). Сервер принимает исключительно одноканальные аудио, поэтому необходимо выбрать один из двух в случае двухканальной записи.

Данный метод возвращает последовательность байтов (**audio_bytes**), частоту дискретизации (**sr**), глубину кодирования (**dtype**) и длительность в секундах (**duration**)

In [12]:
audio_bytes, sr, dtype, duration = load_wav('test_audios/test1.wav', max_duration=-1, channel=0)

Для распознавания используется метод **recognize(audio_bytes, sr, dtype, mode)**, где:
- **audio_bytes** — байты, извлеченные из аудиофайла;
- **sr** — частота дискретизации;
- **dtype** — глубина кодирования (на данный момент поддерживается только int16);
- **mode** — режим распознавания ('short' для Short Mode и 'long' для Long Mode).

<br>

Ответы от сервера:
- для **mode = 'short'** метод возвращает строку **oleg** если запись задержит речь Ассистента Олега, **not_oleg** если запись задержит речь человека или другого автоответчика, либо **None** если запись не содержит речь;

- для **mode = 'long'** метод возвращает **таблицу (объект pandas.DataFrame)** если запись задержит речь и **None** иначе. Таблица содержит начало фраз с речью в секундах **start**, конец **end** и длительность **duration**, а также результаты распознавания **results** (строки 'oleg' или 'not_oleg') для каждой фразы и **confidence** (вероятность, что речь принадлежит Ассистенту Олегу).

#### Пример №1 (Short Mode): запись содержит речь Ассистента Олега

In [13]:
audio_bytes, sr, dtype, duration = load_wav('test_audios/test1.wav', max_duration=-1, channel=0)
results = recognizer.recognize(audio_bytes=audio_bytes, sr=sr, dtype=dtype, mode='short')
print(results)

oleg


#### Пример №2 (Short Mode): запись содержит речь человека

In [14]:
audio_bytes, sr, dtype, duration = load_wav('test_audios/test1.wav', max_duration=-1, channel=1)
results = recognizer.recognize(audio_bytes=audio_bytes, sr=sr, dtype=dtype, mode='short')
print(results)

not_oleg


#### Пример №3 (Short Mode): запись не содержит речь

In [15]:
audio_bytes, sr, dtype, duration = load_wav('test_audios/test4.wav', max_duration=-1, channel=0)
results = recognizer.recognize(audio_bytes=audio_bytes, sr=sr, dtype=dtype, mode='short')
print(results)

Audio data does not contain speech
None


#### Пример №4 (Long Mode): запись содержит речь Ассистента Олега

In [16]:
audio_bytes, sr, dtype, duration = load_wav('test_audios/test2.wav', max_duration=-1, channel=0)
results = recognizer.recognize(audio_bytes=audio_bytes, sr=sr, dtype=dtype, mode='long')
results

,start,end,duration,confidence,results
0,2.7,5.1,2.4,1,oleg
1,5.3,6.3,1.0,1,oleg
2,9.3,16.0,6.7,1,oleg


#### Пример №5 (Long Mode): запись содержит речь человека

In [17]:
audio_bytes, sr, dtype, duration = load_wav('test_audios/test3.wav', max_duration=-1, channel=0)
results = recognizer.recognize(audio_bytes=audio_bytes, sr=sr, dtype=dtype, mode='long')
results

,start,end,duration,confidence,results
0,1.2,1.5,0.3,0.32,not_oleg
1,1.9,3.0,1.1,0.00,not_oleg
2,3.0,3.7,0.7,0.00,not_oleg
3,5.8,6.5,0.7,0.00,not_oleg
4,10.8,11.1,0.3,0.19,not_oleg
5,13.6,14.9,1.3,0.00,not_oleg
6,23.0,24.7,1.7,0.00,not_oleg
7,31.6,33.0,1.4,0.00,not_oleg
8,38.6,40.5,1.9,0.00,not_oleg


#### Пример №6 (Long Mode): запись не содержит речь

In [18]:
audio_bytes, sr, dtype, duration = load_wav('test_audios/test4.wav', max_duration=-1, channel=0)
results = recognizer.recognize(audio_bytes=audio_bytes, sr=sr, dtype=dtype, mode='long')
print(results)

Audio data does not contain speech
None


## Замечание

Примеры POST запросов представлены в **od_client/request_methods** и могут быть переписаны на другом языке программирования, как и метод для загрузки аудиофайлов (например, возможно отправлять на сервер для распознавания байты, полученные с микрофона, а не с файла)